In [ ]:
import spacy

!python -m spacy download pt_core_news_lg
pln = spacy.load('pt_core_news_lg')
pln

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
caminho_arquivo = '/content/drive/MyDrive/TCC - UFMS/CSBC-WICS/Corpus/corpus.csv'
dataframe = pd.read_csv(caminho_arquivo)
# Filtrar o DataFrame para excluir as letras do gênero gospel
dataframe_sem_gospel = dataframe[dataframe['genre'] != 'gospel']
# Exibir as primeiras linhas do DataFrame filtrado
dataframe_sem_gospel.head()

,artist_name,genre,music_title,music_lyric
0,Black Style,axe,Amassa a Latinha Com a Bunda,Eu tô trazendo a novidade em salvador\n(eu tô ...
1,Black Style,axe,Vaco Vuco,Você marcou comigo pra sair\nMas já pensou se ...
2,Black Style,axe,"Tome, Todinha",Que mulher gulosa\nEla entra na roda e não que...
3,Black Style,axe,Toma Que É de Graça,"Toma, toma meu carinho\ntoma meu amor e a minh..."
4,Black Style,axe,Toda de Mel,"Foi esculpida,escolhida,desenhada no papel\nNã..."


In [8]:
from spacy.matcher import Matcher

# Create a Matcher and provide model vocabulary; assign result under the variable 'matcher'
matcher = Matcher(vocab=pln.vocab)

# Define a list with nested dictionaries that contains the pattern to be matched
#Você + Auxiliar + Adjetivo (Feminino)
pronoun_aux_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}] #Você é cheirosa.
#Sujeito + Auxiliar + Adjetivo
sujeito_aux_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': 'AUX'}, {'POS': 'ADJ'}]
#Você + Auxiliar + muito/pouco + Adjetivo
pronoun_aux_adv_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}] # Ela é muito [pouco] inteligente.
#Sujeito + Auxiliar + muito/pouco + Adjetivo
sujeito_aux_adv_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {'POS': 'ADJ'}] # Ela é muito [pouco] inteligente.
#Voce + Auxiliar + Auxiliar + Adjetivo (Feminino)
pronoun_aux_aux_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {'POS': 'AUX'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]
#Sujeito + Auxiliar + Auxiliar + Adjetivo
sujeito_aux_aux_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': 'AUX'}, {'POS': 'AUX'}, {'POS': 'ADJ'}]
#Voce + Auxiliar + Auxiliar + Muito/pouco + Adjetivo (Feminino)
pronoun_aux_aux_adv_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]
#Sujeito + Auxiliar + Auxiliar + muito/pouco + Adjetivo
sujeito_aux_aux_adv_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': 'AUX'}, {'POS': 'ADV', 'OP': '+'}, {'POS': 'ADJ'}]
#Sujeito + Auxiliar + Verbo + Adjetivo
sujeito_aux_verb_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': 'AUX'}, {"POS":"VERB"}, {'POS': 'ADJ'}]
#Sujeito + Auxiliar + Verbo + muito/pouco + Adjetivo
sujeito_aux_verb_adv_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': 'AUX'}, {"POS":"VERB"}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ'}]
#Voce + Auxiliar + Verbo + Adjetivo (Feminino)
pronoun_aux_verb_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {"POS":"VERB"}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}] # Ele está ficando velho.
#Voce + Auxiliar + Verbo + muito/pouco + Adjetivo (Feminino)
pronoun_aux_verb_adv_adj = [{'POS': 'PRON', "MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'AUX'}, {"POS":"VERB"}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]
#Sujeito + Verbo + Auxiliar + Adjetivo
sujeito_verb_aux_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': 'VERB'}, {'POS': 'AUX'}, {'POS': 'ADJ'}]
#Sujeito + Verbo + Auxiliar + muito/pouco + Adjetivo
sujeito_verb_aux_adv_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': 'VERB'}, {'POS': 'AUX'}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ'}]
#Você + Verbo + Auxiliar + Adjetivo (Feminino)
pronoun_verb_aux_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'VERB'}, {'POS': 'AUX'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]
#Você + Verbo + Auxiliar + muito/pouco + Adjetivo (Feminino)
pronoun_verb_aux_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'VERB'}, {'POS': 'AUX'}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]
#Sujeito + Auxiliar(Ser/Estar) + verbo principal no particípio passado
sujeito_aux_verb = [{'LOWER': {'IN': ["você", "vocês","mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': 'AUX'}, {'POS': 'VERB', "MORPH": "Gender=Fem|Number=Sing|VerbForm=Part|Voice=Pass"}]
#Sujeito + Auxiliar(Ser/Estar) + muito/pouco + verbo principal no particípio passado
sujeito_aux_adv_verb = [{'LOWER': {'IN': ["você", "vocês","mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': 'AUX'}, {'POS': "ADV", 'OP': '+'}, {'POS': 'VERB', "MORPH": "Gender=Fem|Number=Sing|VerbForm=Part|Voice=Pass"}]
#Sujeito + Verbo + Adjetivo / LEMMA tornar, ficar, parecer
sujeito_verb_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ'}]
#Sujeito + Verbo + muito/pouco + Adjetivo / LEMMA tornar, ficar, parecer
sujeito_verb_adv_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ'}]
#Voce + Verbo + Adjetivo / LEMMA tornar, ficar, parecer
pronoun_verb_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]
#Voce + Verbo + muito/pouco + Adjetivo / LEMMA tornar, ficar, parecer
pronoun_verb_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]
#Sujeito + Adjetivo
sujeito_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {"POS":"ADJ"}]
#Sujeito + Pronome +  Verbo (LEMMA) + Adjetivo / Ela se tornou bonita
sujeito_pron_verb_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ'}]
#Você +Pronome +  Verbo (LEMMA) + Adjetivo / Você se tornou bonita
pronoun_pron_verb_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]
#Sujeito +Pronome +  Verbo (LEMMA) + Adjetivo / Ela se tornou muito bonita
sujeito_pron_verb_adv_adj = [{'LOWER':  {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ'}]
#Você +Pronome +  Verbo (LEMMA) + Adjetivo / Você se tornou muito bonita
pronoun_pron_verb_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]
#Sujeito + Auxiliar + Pronome +  Verbo (LEMMA) + Adjetivo
sujeito_aux_pron_verb_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': "AUX"}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ'}]
#Voce + Auxiliar + Pronome +  Verbo (LEMMA) + Adjetivo
pronoun_aux_pron_verb_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS': "AUX"}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ' , "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]
#Voce + Auxiliar + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
pronoun_aux_pron_verb_adv_adj = [{'POS': 'PRON',"MORPH":{"IN":['Case=Nom|Number=Sing|Person=3|PronType=Prs']}},{'POS':'AUX'}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]
#Sujeito + Auxiliar + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
sujeito_aux_pron_verb_adv_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': "AUX"}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ'}]
#Sujeito + Verbo + Pronome + Verbo (LEMMA) + Adjetivo
sujeito_verb_pron_verb_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': "VERB"}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ'}]
# Você + Verbo + Pronome +  Verbo (LEMMA) + Adjetivo
pronoun_verb_pron_verb_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS':'VERB'}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]
# Sujeito + Verbo + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
sujeito_verb_pron_verb_adv_adj = [{'LOWER': {'IN': ["mulher", "mulheres", "ela", "elas", "menina", "meninas", "garota", "garotas", "senhora", "senhoras", "senhorita", "senhoritas", "moça", "moças", "donzela", "donzelas", "maria", "dama", "damas", "rainha", "rainhas", "esposa", "esposas", "namorada", "namoradas"]}}, {'POS': "VERB"}, {'POS':'PRON'},{'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}},{'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ'}]
# Você + Verbo + Pronome +  Verbo (LEMMA) + muito/pouco + Adjetivo
pronoun_verb_pron_verb_adv_adj = [{'POS': 'PRON',"MORPH": {"IN": ['Case=Nom|Number=Sing|Person=3|PronType=Prs']}}, {'POS':'VERB'}, {'POS':'PRON'}, {'POS': 'VERB', "LEMMA":{"IN":["parecer", "ficar", "tornar"]}}, {'POS': "ADV", 'OP': '+'}, {'POS': 'ADJ', "MORPH": {"IN":["Gender=Fem|Number=Plur", "Gender=Fem|Number=Sing"]}}]

# Add the patterns to the matcher under different names
matcher.add('pronoun+aux+adj', patterns=[pronoun_aux_adj])
matcher.add('sujeito_aux_adj', patterns=[sujeito_aux_adj])
matcher.add('pronoun+aux+adv+adj', patterns=[pronoun_aux_adv_adj])
matcher.add('sujeito_aux_adv_adj', patterns=[sujeito_aux_adv_adj])
matcher.add('pronoun_aux_aux_adj ', patterns=[pronoun_aux_aux_adj])
matcher.add('sujeito_aux_aux_adj', patterns=[sujeito_aux_aux_adj])
matcher.add('pronoun_aux_aux_adv_adj', patterns=[pronoun_aux_aux_adv_adj])
matcher.add('sujeito_aux_aux_adv_adj', patterns=[sujeito_aux_aux_adv_adj])
matcher.add('sujeito_aux_verb_adj', patterns=[sujeito_aux_verb_adj])
matcher.add('sujeito_aux_verb_adv_adj', patterns=[sujeito_aux_verb_adv_adj])
matcher.add('pronoun_aux_verb_adj', patterns=[pronoun_aux_verb_adj])
matcher.add('pronoun_aux_verb_adv_adj', patterns=[pronoun_aux_verb_adv_adj])
matcher.add('sujeito_verb_aux_adj', patterns=[sujeito_verb_aux_adj])
matcher.add('sujeito_verb_aux_adv_adj', patterns=[sujeito_verb_aux_adv_adj])
matcher.add('sujeito_adj', patterns=[sujeito_adj])
matcher.add('pronoun_verb_adv_adj', patterns=[pronoun_verb_adv_adj])
matcher.add('pronoun_verb_adj', patterns=[pronoun_verb_adj])
matcher.add('sujeito_verb_adv_adj', patterns=[sujeito_verb_adv_adj])
matcher.add('sujeito_verb_adj', patterns=[sujeito_verb_adj])
matcher.add('sujeito_aux_adv_verb', patterns=[sujeito_aux_adv_verb])
matcher.add('sujeito_aux_verb', patterns=[sujeito_aux_verb])
matcher.add('sujeito_pron_verb_adj', patterns=[sujeito_pron_verb_adj])
matcher.add('pronoun_pron_verb_adj', patterns=[pronoun_pron_verb_adj])
matcher.add('sujeito_pron_verb_adv_adj', patterns=[sujeito_pron_verb_adv_adj])
matcher.add('pronoun_pron_verb_adv_adj', patterns=[pronoun_pron_verb_adv_adj])
matcher.add('pronoun_aux_pron_verb_adj', patterns=[pronoun_aux_pron_verb_adj])
matcher.add('sujeito_aux_pron_verb_adj', patterns=[sujeito_aux_pron_verb_adj])
matcher.add('pronoun_aux_pron_verb_adv_adj', patterns=[pronoun_aux_pron_verb_adv_adj])
matcher.add('sujeito_aux_pron_verb_adv_adj', patterns=[sujeito_aux_pron_verb_adv_adj])
matcher.add('sujeito_verb_pron_verb_adj', patterns=[sujeito_verb_pron_verb_adj])
matcher.add('pronoun_verb_pron_verb_adj', patterns=[pronoun_verb_pron_verb_adj])
matcher.add('pronoun_verb_pron_verb_adv_adj', patterns=[pronoun_verb_pron_verb_adv_adj])
matcher.add('sujeito_verb_pron_verb_adv_adj', patterns=[sujeito_verb_pron_verb_adv_adj])


In [9]:
resultados = []

for index, row in dataframe_sem_gospel.iterrows():
    sentenca = str(row['music_lyric'])
    doc = pln(sentenca)

    results = matcher(doc, as_spans=True)

    for result in results:
      if len(result) > 0:
        #print(pln.vocab[result.label].text, '\t', result)
        resultados.append({'Padrão': pln.vocab[result.label].text, 'Frase': result, 'Letra_Inteira': sentenca})

resultados_df = pd.DataFrame(resultados)
resultados_df.head()
        #displacy.render(doc, style='dep', options={'compact': True, 'distance': 100}, jupyter=True)

,Padrão,Frase,Letra_Inteira
0,sujeito_adj,"(mulher, gulosa)",Que mulher gulosa\nEla entra na roda e não que...
1,sujeito_aux_adj,"(ela, é, cheirosa)","Ai ela é cheirosa, ela é charmosa, ela é um te..."
2,sujeito_aux_adj,"(ela, é, charmosa)","Ai ela é cheirosa, ela é charmosa, ela é um te..."
3,sujeito_aux_adj,"(Ela, é, cheirosa)","Ai ela é cheirosa, ela é charmosa, ela é um te..."
4,sujeito_aux_adj,"(ela, é, charmosa)","Ai ela é cheirosa, ela é charmosa, ela é um te..."


In [10]:
from google.colab import files

# Supondo que 'resultados' é uma lista de dicionários ou uma estrutura adequada para criar um DataFrame
resultados_df = pd.DataFrame(resultados)

# Salvar o DataFrame como um arquivo CSV
resultados_df.to_csv('busca_com_POS.csv', index=False)

# Fazer o download do arquivo CSV
files.download('busca_com_POS.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>